## Unit 08. Association Rule Learning 課堂作業
- 課程編號: CHEXXXX
- 學年度: 114下
- 上課時間: 每週四 09:00-12:00
-
- 指導教授: ＯＯＯ 教授
- 學生姓名: ＯＯＯ
- 學號: m12345678
- email帳號: fcu.m12345678@gmail.com

---
### 環境設定

In [ ]:
from pathlib import Path
import os

# ========================================
# 路徑設定 (兼容 Colab 與 Local)
# ========================================
UNIT_OUTPUT_DIR = 'P2_Unit08_HW'
SOURCE_DATA_DIR = 'catalyst_formulations'

try:
  from google.colab import drive
  IN_COLAB = True
  print("✓ 偵測到 Colab 環境，準備掛載 Google Drive...")
  drive.mount('/content/drive', force_remount=True)
except ImportError:
  IN_COLAB = False
  print("✓ 偵測到 Local 環境")
try:
  shortcut_path = '/content/CHE-AI-COURSE'
  os.remove(shortcut_path)
except FileNotFoundError:
  pass

if IN_COLAB:
  source_path = Path('/content/drive/My Drive/Colab Notebooks/CHE-AI-COURSE')
  os.symlink(source_path, shortcut_path)
  shortcut_path = Path(shortcut_path)
  if source_path.exists():
    NOTEBOOK_DIR = shortcut_path / 'Part_2' / 'Unit08'
    OUTPUT_DIR = NOTEBOOK_DIR / 'outputs' / UNIT_OUTPUT_DIR
    DATA_DIR = NOTEBOOK_DIR / 'data' / SOURCE_DATA_DIR
    FIG_DIR = OUTPUT_DIR / 'figs'
  else:
    print(f"⚠️ 找不到路徑雲端CHE-AI-COURSE路徑，請確認自己的雲端資料夾是否正確")
  
else:
  NOTEBOOK_DIR = Path.cwd()
  OUTPUT_DIR = NOTEBOOK_DIR / 'outputs' / UNIT_OUTPUT_DIR
  DATA_DIR = NOTEBOOK_DIR / 'data' / SOURCE_DATA_DIR
  FIG_DIR = OUTPUT_DIR / 'figs'

NOTEBOOK_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
DATA_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

print(f"\n✓ Notebook工作目錄: {NOTEBOOK_DIR}")
print(f"✓ 數據來源目錄: {DATA_DIR}")
print(f"✓ 結果輸出目錄: {OUTPUT_DIR}")
print(f"✓ 圖檔輸出目錄: {FIG_DIR}")

---
### **I. 練習題: 催化劑配方關聯規則挖掘**
 - 使用下面化工催化劑配方模擬數據集。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Association Rule Mining 套件
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth

# 網絡圖視覺化
import networkx as nx

# 設定隨機種子以確保結果可重現
SEED = 42
np.random.seed(SEED)

# 設定matplotlib中文顯示
plt.rcParams['axes.unicode_minus'] = False

print("✓ 套件載入完成")

# ========================================
# 模擬催化劑配方數據集
# ========================================
# 定義項目
metals = ['Metal_Pt', 'Metal_Pd', 'Metal_Ni', 'Metal_Ru', 'Metal_Rh']
supports = ['Support_Al2O3', 'Support_SiO2', 'Support_ZrO2', 'Support_TiO2']
promoters = ['Promoter_La', 'Promoter_Ce', 'Promoter_K', 'Promoter_Na']
binders = ['Binder_Clay', 'Binder_Silica', 'Binder_Alumina']
calcination_temps = ['Calc_Low', 'Calc_Medium', 'Calc_High']
reduction_temps = ['Red_Low', 'Red_Medium', 'Red_High']

# 生成 250 筆催化劑配方交易
n_transactions = 250
transactions = []

print(f"\n生成 {n_transactions} 筆催化劑配方數據...")

for i in range(n_transactions):
    transaction = []
    
    # 每個配方包含 1-2 種金屬
    if np.random.rand() < 0.4:  # 40% 機率選擇 Pt
        metal = 'Metal_Pt'
    else:
        metal = np.random.choice(metals)
    transaction.append(metal)
    
    # 可能添加第二種金屬
    if np.random.rand() < 0.3:
        second_metal = np.random.choice([m for m in metals if m != metal])
        transaction.append(second_metal)
    
    # 1 種載體 (如果是 Pt，更傾向選擇 Al2O3)
    if metal == 'Metal_Pt' and np.random.rand() < 0.65:
        support = 'Support_Al2O3'
    else:
        support = np.random.choice(supports)
    transaction.append(support)
    
    # 0-2 種助劑
    n_promoters = np.random.choice([0, 1, 2], p=[0.2, 0.5, 0.3])
    if n_promoters > 0:
        selected_promoters = np.random.choice(promoters, n_promoters, replace=False)
        transaction.extend(selected_promoters)
    
    # 50% 機率添加黏結劑
    if np.random.rand() < 0.5:
        binder = np.random.choice(binders)
        transaction.append(binder)
    
    # 1 種焙燒溫度
    calcination = np.random.choice(calcination_temps)
    transaction.append(calcination)
    
    # 1 種還原溫度
    reduction = np.random.choice(reduction_temps)
    transaction.append(reduction)
    
    # 基於條件添加品質指標
    # 規則1: {Metal_Pt, Support_Al2O3} → {High_Activity}
    if 'Metal_Pt' in transaction and 'Support_Al2O3' in transaction and np.random.rand() < 0.8:
        transaction.append('High_Activity')
    elif np.random.rand() < 0.3:
        transaction.append('High_Activity')
    
    # 規則2: {Promoter_La, Calc_High} → {High_Stability}
    if 'Promoter_La' in transaction and 'Calc_High' in transaction and np.random.rand() < 0.75:
        transaction.append('High_Stability')
    elif np.random.rand() < 0.25:
        transaction.append('High_Stability')
    
    # 規則3: {Red_Medium, Binder_Clay} → {High_Selectivity}
    if 'Red_Medium' in transaction and 'Binder_Clay' in transaction and np.random.rand() < 0.7:
        transaction.append('High_Selectivity')
    elif np.random.rand() < 0.2:
        transaction.append('High_Selectivity')
    
    transactions.append(transaction)

print(f"✓ 生成 {len(transactions)} 筆配方數據")
print(f"\n前5筆配方範例:")
for i, trans in enumerate(transactions[:5], 1):
    print(f"  配方 {i}: {trans}")

### **1. 數據探索與分析**
- 使用 `TransactionEncoder` 將交易數據轉換為 DataFrame
- 計算各項目的支持度，並繪製前20項的支持度條形圖
- 分析數據集中項目的分布特徵
- 保存轉換後的數據至 CSV 文件

In [ ]:
# 請在此處完成程式碼

---
### **2. Apriori 演算法實作**

使用 Apriori 演算法從催化劑配方數據中挖掘關聯規則。

#### **2.1 頻繁項集挖掘**
- 使用 `apriori()` 函數挖掘頻繁項集
- 設定 `min_support=0.1` (最小支持度10%)
- 顯示前10個頻繁項集
- 分析頻繁項集的長度分布

In [ ]:
# 請在此處完成程式碼

#### **2.2 關聯規則生成與評估**
- 使用 `association_rules()` 函數生成關聯規則
- 設定評估指標 `metric='lift'`，閾值 `min_threshold=1.2`
- 計算規則數量
- 顯示前10條規則，包含 support, confidence, lift 等指標
- 分析規則的品質分布

In [ ]:
# 請在此處完成程式碼

#### **2.3 規則視覺化**
- 繪製 Support vs Confidence 散點圖 (使用 Lift 作為顏色映射)
- 繪製 Support vs Lift 散點圖
- 選擇 Lift > 1.5 且 Confidence > 0.6 的高品質規則
- 使用 NetworkX 繪製規則網絡圖 (前15條規則)
- 保存所有圖表

In [ ]:
# 請在此處完成程式碼

---
### **3. FP-Growth 演算法實作**

使用 FP-Growth 演算法從相同的催化劑配方數據中挖掘關聯規則。

#### **3.1 頻繁項集挖掘**
- 使用 `fpgrowth()` 函數挖掘頻繁項集
- 使用與 Apriori 相同的 `min_support=0.1`
- 顯示前10個頻繁項集
- 比較 FP-Growth 與 Apriori 挖掘的頻繁項集數量

In [ ]:
# 請在此處完成程式碼

#### **3.2 關聯規則生成與評估**
- 使用 `association_rules()` 函數生成關聯規則
- 設定評估指標 `metric='lift'`，閾值 `min_threshold=1.2`
- 計算規則數量
- 顯示前10條規則
- 比較 FP-Growth 與 Apriori 生成的規則數量

In [ ]:
# 請在此處完成程式碼

#### **3.3 規則視覺化**
- 繪製 Support vs Confidence 散點圖 (使用 Lift 作為顏色映射)
- 選擇 Lift > 1.5 且 Confidence > 0.6 的高品質規則
- 使用 NetworkX 繪製規則網絡圖 (前15條規則)
- 保存所有圖表

In [ ]:
# 請在此處完成程式碼

---
### **4. Apriori vs FP-Growth 比較分析**

比較兩種演算法的性能與結果。

#### **4.1 執行效率比較**
- 使用 `time` 模組測量兩種演算法的執行時間
- 比較頻繁項集挖掘階段的執行時間
- 比較關聯規則生成階段的執行時間
- 繪製執行時間對比條形圖
- 分析效率差異原因

In [ ]:
# 請在此處完成程式碼

#### **4.2 規則品質比較**
- 比較兩種演算法生成的規則總數
- 比較高品質規則 (Lift > 1.5 且 Confidence > 0.6) 的數量
- 繪製兩種演算法的 Lift 分布直方圖
- 繪製兩種演算法的 Confidence 分布直方圖
- 找出僅在 Apriori 或僅在 FP-Growth 中出現的規則 (如果有差異)
- 分析規則品質差異

In [ ]:
# 請在此處完成程式碼

#### **4.3 化工應用分析**
- 從兩種演算法的高品質規則中，找出與品質指標相關的規則
  - 與 `High_Activity` 相關的規則
  - 與 `High_Stability` 相關的規則
  - 與 `High_Selectivity` 相關的規則
- 分析這些規則的化工意義
- 提出催化劑配方優化建議
- 討論哪種演算法更適合此類化工配方挖掘應用

In [ ]:
# 請在此處完成程式碼

---
### **5. 總結與心得**

請根據本次作業的實作經驗，回答以下問題：

#### **5.1 學習收穫**
1. **Apriori 演算法的優缺點是什麼？**
   - (請在此回答)

2. **FP-Growth 演算法的優缺點是什麼？**
   - (請在此回答)

3. **在什麼情況下應該選擇 Apriori？什麼情況下應該選擇 FP-Growth？**
   - (請在此回答)

4. **Support, Confidence, Lift 這三個指標各自代表什麼意義？在實際應用中應該如何設定閾值？**
   - (請在此回答)

5. **關聯規則學習在化工領域還有哪些潛在應用？**
   - (請在此回答)

#### **5.2 實作心得**
請分享你在完成本次作業過程中的心得，包括：
- 遇到的主要挑戰
- 解決問題的方法
- 對關聯規則學習的理解
- 對未來學習的規劃

(請在此回答)

---
### 💭 思考題

1. **Support, Confidence, Lift 這三個指標如何相互影響？**

2. **為什麼在化工配方挖掘中，Lift 比 Confidence 更重要？**

3. **如何判斷一條關聯規則是否具有實際應用價值？**

4. **Apriori 和 FP-Growth 的時間複雜度有何差異？為什麼？**

5. **如果數據集中有大量低頻項目，應該如何處理？**

6. **關聯規則挖掘的結果如何與領域專家知識結合？**

7. **在實際化工應用中，如何設定合理的 min_support 和 min_confidence？**

8. **關聯規則能否用於因果推斷？為什麼？**

---
# 想對老師說的話

---
### **作業完成**

**提交要求：**
1. 完成所有程式碼實作
2. 確保所有圖表都已保存至 `outputs` 資料夾
3. 回答所有問題
4. 提交完整的 `.ipynb` 檔案與 `outputs` 資料夾

**評分標準：**
- 程式碼正確性與完整性 (40%)
- 數據分析與視覺化品質 (30%)
- 演算法比較分析深度 (20%)
- 問題回答與心得 (10%)

祝作業順利完成！